<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Clustering con kmeans

Crear un modelo de segmentación para la estimación de los tipos de clientes según su hábito de compras, ingresos, edad, etc<br>
v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle"></div>

In [ ]:
if os.access('Cust_Segmentation.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Cust_Segmentation.csv > Cust_Segmentation.csv
    else:
        !wget Mall_Customers.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/Cust_Segmentation.csv

### `Cust_Segmentation.csv`:
El dataset **`Cust_Segmentation.csv`** contiene datos sobre personas, sus hábitos de consumo, edad, etc, los cuales pueden ser utilizados para dirigir una campaña de publicidad o de ventas.<br>
[Dataset source](https://www.kaggle.com/gangliu/custdatasets)
- **CustomerID** --> id del comprador, ejemplo 5
- **Age** --> edad
- **Edu** --> Nivel de educación
- **Years Emplyed** --> Años que lleva trabajando
- **Income** --> ingreso de dinero anual de la persona en miles de dolares
- **Card Debt** --> Débitos de la tarjeta (gastos)
- **Other Debt** --> Otros gastos
- **Defaulted** --> Deudor (0 --> No), (1 --> Si)
- **Adress** --> Dirección
- **DebtIncomeRatio** --> que tan gastador es la persona

In [ ]:
df = pd.read_csv("Cust_Segmentation.csv")

# Procesar datos
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle"></div>

In [ ]:
# inspeccione el dataset, visualice las columnas que posee y sus primeras filas
# Ver que columnas son categóricas y numéricas
df.head()

In [ ]:
# Los algoritmos de clustering no funcionan bien con las variables categóricas, 
# obtener un nuevo dataset (df2) sin la columna Address, Defaulted y Customer Id
# La coulmna "Edu" se puede considerar como numérica no categórica, porque
# cuanto más alto su valor mayor nivel de educación tiene la persona, por lo que
# los "números" tiene un significado
df2 = df.drop(['Customer Id', 'Defaulted', 'Address'], axis=1)

In [ ]:
# Realizar una inspeccion del dataset en búsqueda de elementos faltantes
des = df2.describe()
des.loc['Nan'] = df2.isna().sum()
des.loc['%Nan'] = (df2.isna().mean())*100
des

In [ ]:
# ¿Cuántas filas/datos hay para analizar?
print('Datos disponibles para analizar: ', df2.shape[0])

# Explorar datos
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle"></div>

In [ ]:
# Observar todas las relaciones entre todos los features, utilizar pairplot
sns.pairplot(df2)

A simple vista es muy dificil distingir grupos relacionando solo dos variables. Confiaremos en el algoritmo de clustering y utilizaremos todo los features de entrada

# Entrenar modelo
<div align="center"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle"></div>

In [ ]:
# Crear un numpy array "X" con los features de entrada
# Se analizó si era conveniente realizar la normalización de datos, cuando se
# ensayó se obtuvo un peor resultado. Cuando se realiza clustering hay que tener
# cuidado que el proceso de normalización puede afectar la distribución
# de los datos y por lo tanto alterar el resultado (es cuestión de probar)
# from sklearn.preprocessing import StandardScaler
# df3 = df2.copy()
# X = StandardScaler().fit_transform(df3)
X = df2.values

#### Crear un modelo de segmentación con Kmeans
Parámetros
- n_clusters --> (k) número de clusters/grupos (defecto 5)
- init --> método utilizado para determinar donde comienzan los clusters
 - k-means++ --> mecanismo inteligente para determinar el comienzo (defecto)
 - random --> los centros se determinan aleatoriamente
- max_iter --> cantidad de iteración (defecto 300)

In [ ]:
# Almuno: Entrenar diferentes modelos de Kmeans en un rango de cluster (2, 10)
# Conservar el resultado de "inertial" para utilizar como métrica de selccion
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def find_best_model(X, max_clusters=10):

    n_clusters = list(range(2, max_clusters+1))
    ensayos = len(n_clusters)

    ssd = np.zeros((ensayos))
    score = np.zeros((ensayos))

    for i in range(ensayos): 
        kmeans = KMeans(n_clusters=n_clusters[i], init="k-means++", random_state=0)
        kmeans.fit(X)
        ssd[i] = kmeans.inertia_
        score[i] = silhouette_score(X, kmeans.labels_)

    # Dibujar la evolucion del error (inertia) y deteminar el 
    # punto de quiebre (elbow point) para deteminar el mejor k
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.plot(n_clusters, ssd/ssd.max(), label='error')
    ax.plot(n_clusters, score, label='score')
    ax.grid('dashed')
    ax.legend()
    plt.show()
    best_cluster = n_clusters[score.argmax()]
    print('El mejor resultado se consigue con clusters=', best_cluster)
    return best_cluster

In [ ]:
best_cluster = find_best_model(X)

In [ ]:
# No se obtuvo un resultado muy bueno, el punto de codo (elbow) con K=4 no coincide
# con el mejor punto de score del modelo
df4 = df2.drop(['Years Employed', 'Age'], axis=1)
X2 = df4.values

best_cluster = find_best_model(X2)

Reduciendo un poco los features se obtuvo un mejor modelo, aún el punto de codo no coincide con k=4, pero se ve claramente que ese el punot indicado

In [ ]:
# Entrenar el modelo con el mejor K (el punto de codo)
kmeans = KMeans(n_clusters=4, init="k-means++", random_state=0)
kmeans.fit(X2)
labels = kmeans.predict(X2)

In [ ]:
# Sumar la columna de labels con el nombre de custseg (customer segmentation)
# a su dataset
df4['custseg'] = labels

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy) del modelo
from sklearn.metrics import silhouette_score
silhouette_score (X2, kmeans.labels_)

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
# Observar todas las relaciones entre todos los features, utilizar pairplot
sns.pairplot(df4, hue='custseg', palette='bright')

In [ ]:
# Observar la relación entre el ingreso (income) y el débito
# pero ahora coloreando con "hue" los puntos según su clase de custseg
sns.scatterplot(data=df4, x='Income', y='Card Debt', hue='custseg', palette='bright')
plt.show()

Se puede ve que no existe una relación lineal entre ingreso y gasto

In [ ]:
# Observar la relación entre el nivel de educación (Edu) y el débito
# pero ahora coloreando con "hue" los puntos según su clase de custseg
sns.scatterplot(data=df4, x='Edu', y='Card Debt', hue='custseg', palette='bright')
plt.show()

Es muy dificil de distingir los clusters al comparar "Edu" y "Débitos", utilizemos un gráfico 3D para visualizar tanto el "Income" como la educación "Edu" respecto a los débitos de tarjeta

In [ ]:
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot(projection='3d')

n_clusters = df4['custseg'].unique()

for cluster in n_clusters:
    data = df4[df4['custseg'] == cluster]
    ax.scatter(data['Income'], data['Edu'], data['Card Debt'], label='cluster ' + str(cluster))

ax.set_xlabel('Ingresos')
ax.set_ylabel('Educación')
ax.set_zlabel('Débito')
ax.legend()
ax.view_init(45, 30)
plt.show()

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

La conclusión es que se puede observar en el último gráfico que a mayor ingreso las personas gastan más, pero es notoria que a mayor nivel de educación pareciera realizarse menos gastos (¿o capaz haya menos información para educación más elevada?

In [ ]:
# Analizar la distribución de la educación en el dataset
fig = plt.figure(figsize=(16, 9))
ax = fig.add_subplot()
sns.boxplot(x=df4['Edu'], ax=ax)
ax.grid('dashed')

Correcto! La mayor cantidad de información se encuentra para los niveles de estudio intermedio y bajo, es por eso que no hay suficiente información para los estudios elevados y confirma el resultado observado